In [1]:
# Importing Pandas an Numpy Libraries to use on manipulating our Data
import pandas as pd
import numpy as np

# To Preproccesing our data
from sklearn.preprocessing import LabelEncoder

# To fill missing values
from sklearn.impute import SimpleImputer

# To Split our train data
from sklearn.model_selection import train_test_split

# To Visualize Data
import matplotlib.pyplot as plt
import seaborn as sns

# To Train our data
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB

# To evaluate end result we have 
from sklearn.metrics import accuracy_score, confusion_matrix,mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

In [2]:
#We are importing our Data with Pandas Library
# We use "Coronary_artery.csv" 
df = pd.read_csv("../Data/AdSmartABdata.csv")
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [3]:
clean_data = df[df['yes'] == 1].append(df[df['no'] == 1]) 
 

<ipython-input-3-37a9331583fc>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clean_data = df[df['yes'] == 1].append(df[df['no'] == 1])


In [4]:
clean_data.isna().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

In [5]:
clean_data['date'] = pd.to_datetime(df['date'], errors='coerce')

clean_data['date_of_week'] = clean_data['date'].dt.day_name()

In [6]:
clean_data.drop(['no'], axis=1,inplace=True)
clean_data.rename(columns={'yes': 'result'},inplace=True)

In [7]:
exposed_group = clean_data[clean_data['experiment'] == 'exposed']
control_group = clean_data[clean_data['experiment'] == 'control']

In [8]:
lb = LabelEncoder()
clean_data['experiment'] = lb.fit_transform(clean_data['experiment'])
clean_data['browser'] = lb.fit_transform(clean_data['browser'])
clean_data['device_make'] = lb.fit_transform(clean_data['device_make'])
clean_data['date_of_week'] = lb.fit_transform(clean_data['date_of_week'])

In [9]:
clean_data['row_id'] = clean_data.index

In [10]:
clean_data.dropna(inplace=True)
clean_data = shuffle(clean_data)

In [11]:
clean_data = clean_data[['row_id', 'device_make', 'platform_os', 'browser' ,'date_of_week', 'experiment','result']]
clean_data

,row_id,device_make,platform_os,browser,date_of_week,experiment,result
7510,7510,13,6,1,6,0,0
5978,5978,78,6,2,0,0,0
6833,6833,13,6,1,0,1,1
2537,2537,54,6,7,0,0,1
728,728,13,6,2,3,1,0
...,...,...,...,...,...,...,...
2989,2989,13,6,1,0,1,0
7622,7622,89,6,4,4,1,0
6656,6656,26,6,1,0,1,0
4109,4109,42,6,7,0,0,1


In [12]:
# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(clean_data.loc[:, clean_data.columns != 'result'], clean_data['result'],
    test_size=0.1)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.2) # 0.25 x 0.8 = 0.2

In [13]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(894, 6)
(125, 6)
(224, 6)
(894,)
(125,)
(224,)
